In [2]:
from lxml import html
import requests, re, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random

In [3]:
dictionary =  {'Name': ["Clay Sowell", "Test 2", "Test 3"], 'AB': [57, 3, 4], 'Hits': [23, 1, 2], '2B':[1, 1, 0], '3B':[1, 0, 0], 'HR':[6, 0, 2], 'K':[9, 2, 2], 'Walks':[2, 0, 1], 'HBP':[4, 0, 0]}
df = pd.DataFrame(dictionary)
df.head()

,Name,AB,Hits,2B,3B,HR,K,Walks,HBP
0,Clay Sowell,57,23,1,1,6,9,2,4
1,Test 2,3,1,1,0,0,2,0,0
2,Test 3,4,2,0,0,2,2,1,0


In [4]:
df2 = df
df2['1B'] = 0
df2.head()

,Name,AB,Hits,2B,3B,HR,K,Walks,HBP,1B
0,Clay Sowell,57,23,1,1,6,9,2,4,0
1,Test 2,3,1,1,0,0,2,0,0,0
2,Test 3,4,2,0,0,2,2,1,0,0


In [5]:
curr_thesh = 0
for i, row in df.iterrows():
    curr_thresh = 0
    df2['1B'][i] = df['Hits'][i] - df['2B'][i] - df['3B'][i] - df['HR'][i] + df['Walks'][i] + df['HBP'][i]    
    curr_thresh = ((df['AB'][i]-df['Hits'][i]-df['Walks'][i]-df['HBP'][i])/1.0)/df['AB'][i]
    df2.loc[df2.index[i], 'out_threshold'] = curr_thresh
    curr_thresh += df['1B'][i]/df['AB'][i]
    df2.loc[df2.index[i], '1B_threshold'] = curr_thresh
    curr_thresh += df['2B'][i]/df['AB'][i]
    df2.loc[df2.index[i], '2B_threshold'] = curr_thresh
    curr_thresh += df['3B'][i]/df['AB'][i]
    df2.loc[df2.index[i], '3B_threshold'] = curr_thresh
    curr_thresh += df['HR'][i]/df['AB'][i]
    df2.loc[df2.index[i], 'HR_threshold'] = curr_thresh

df2.head()

/var/folders/p_/1rk5yqrj4d1ffpmr_1zh3qy80000gn/T/ipykernel_92233/1416269239.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['1B'][i] = df['Hits'][i] - df['2B'][i] - df['3B'][i] - df['HR'][i] + df['Walks'][i] + df['HBP'][i]


,Name,AB,Hits,2B,3B,HR,K,Walks,HBP,1B,out_threshold,1B_threshold,2B_threshold,3B_threshold,HR_threshold
0,Clay Sowell,57,23,1,1,6,9,2,4,21,0.491228,0.859649,0.877193,0.894737,1.0
1,Test 2,3,1,1,0,0,2,0,0,0,0.666667,0.666667,1.000000,1.000000,1.0
2,Test 3,4,2,0,0,2,2,1,0,1,0.250000,0.500000,0.500000,0.500000,1.0


In [6]:
def run_poss():
    game_state = dict({'1B': 0, '2B': 0, '3B':0, 'Outs': 0, 'Runs_Jumbos':0, 'Tufts Batter Next':0, 'Inning':1})
    for i in range(1,10):
        # For each inning, reset the out count and bases
        game_state['Outs'] = 0
        game_state['1B'] = 0
        game_state['2B'] = 0
        game_state['3B'] = 0
        
        while (game_state['Outs'] < 3):
            rand = (random.randint(1,10000))/10000.0
            if (rand < df2['out_threshold'][game_state['Tufts Batter Next']]):
                game_state['Outs'] = game_state['Outs']+1
            elif (rand < df2['1B_threshold'][game_state['Tufts Batter Next']]):
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['3B'] = game_state['2B']
                game_state['2B'] = game_state['1B']
                game_state['1B'] = 1
            elif (rand < df2['2B_threshold'][curr_batter]):
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['Runs_Jumbos'] += game_state['2B']
                game_state['3B'] = game_state['1B']
                game_state['2B'] = 1
                game_state['1B'] = 0
            elif (rand < df2['3B_threshold'][game_state['Tufts Batter Next']]):
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['Runs_Jumbos'] += game_state['2B']
                game_state['Runs_Jumbos'] += game_state['1B']
                game_state['3B'] = 1
                game_state['2B'] = 0
                game_state['1B'] = 0
            # Else - home run
            else:
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['Runs_Jumbos'] += game_state['2B']
                game_state['Runs_Jumbos'] += game_state['1B']
                game_state['Runs_Jumbos'] += 1

            game_state['Tufts Batter Next'] = (game_state['Tufts Batter Next']+1)%(len(df2))

    return game_state['Runs_Jumbos']

In [7]:
# Define Variables
num_iterations = 1000
game_state = dict({'1B': 0, '2B': 0, '3B':0, 'Outs': 0, 'Runs_Jumbos':0, "Inning":1, 'Tufts Batter Next':0})

wins_bos = []

def run_montecarlo(num_iter):
    runs = []
    for i in range(num_iter):
        runs.append(run_poss())
    
    return ("Result of montecarlo simulations with batting order is an average of " + str(round(sum(runs)/float(len(runs)), 2)) + " runs.")
            
print(run_montecarlo(num_iterations))

NameError: name 'curr_batter' is not defined

In [ ]:
# TODO - Have one inning, need to run 9 times and aggregate runs scored


        

### Notes:

I assume that Jumbos start each inning -> not hard to change, but while loop continuing game should be altered

Similarly, we should add a way to pick diff pitchers for each time, would have to factor them in somehow to weight performance... or a way for the user to change opposition batting order...


In [194]:
# Working to add web scrapping

# Check beautiful soup for another, more advanced data scraping stat..

from lxml import html
import requests, re, math

# Site to begin scraping
page = 'https://nescac.com/teamstats.aspx?path=baseball&year=2022&school=tufts'


# Scrape start page into tree
result = requests.get(page)
tree = html.fromstring(result.content)


ParserError: Document is empty

In [195]:
# pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [196]:
# Importing the required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [208]:
# Downloading contents of the web page
url = "https://nescac.com/teamstats.aspx?path=baseball&year=2022&school=tufts"
data = requests.get(url, headers={'User-Agent':'test'}).text

In [209]:
data

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide baseball sport baseball"> \r\n    <head><link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700;900&amp;display=swap" /><link rel="stylesheet" href="https://dbukjj6eu5tsf.cloudfront.net/assets.sidearmsports.com/adaptive/css/normalize.min.css" /><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=637859610151777375" /><link rel="apple-touch-icon" href="/images/logos/site/site.png" /><meta charset="utf-8" /><meta name="viewport" content="width=device-width" /><link rel="apple-touch-icon" href="/images/logos/site/site.png" />\t\t\t\t\t\t\r\n\t\t\t<meta name="theme-color" content="#006f53">\r\n\t\t\r\n\r\n<link rel="stylesheet" href="https://fonts.sidearmsports.com/sidearm/sidearm_font.css"/>\r\n<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/slick-carousel/1.5.9/slick.min.css" />\r\n<link rel="stylesheet" href="https://fonts.sidearmsports.com/sid

In [1]:
# Creating BeautifulSoup object
soup = BeautifulSoup(data, 'html.parser')
soup.contents.text

NameError: name 'BeautifulSoup' is not defined

In [211]:
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['sidearm-table']
['sidearm-table']
['sidearm-table']


In [231]:
hitting = soup.find("section", {"id": "hitting"})
hitting

<section id="hitting">
<h3 class="sub-heading">Hitting</h3>
<div class="sidearm-table-overflow">
<table class="sidearm-table" sortable-table="">
<caption class="hide">Hitting Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-hide" scope="col">Player</th>
<th scope="col">AVG</th>
<th class="text-no-wrap" scope="col">GP-GS</th>
<th scope="col">AB</th>
<th scope="col">R</th>
<th scope="col">H</th>
<th scope="col">2B</th>
<th scope="col">3B</th>
<th scope="col">HR</th>
<th scope="col">RBI</th>
<th scope="col">TB</th>
<th scope="col">SLG%</th>
<th scope="col">BB</th>
<th scope="col">HBP</th>
<th scope="col">SO</th>
<th scope="col">GDP</th>
<th scope="col">OB%</th>
<th scope="col">SF</th>
<th scope="col">SH</th>
<th class="text-no-wrap" scope="col">SB-ATT</th>
</tr>
</thead>
<tbody data-bind="foreach: players">
<!-- ko if: hitting_stats -->
<tr>
<th class="text-no-wrap" data-bind="text: name" scope="row"></th>
<td class="text-center" data-bind="text: hitting_stats.bat

In [233]:
# sidearm-table
#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find_all('text-no-wrap')
table

# for row in table.find_all('td'):
#     print(row)
#     print(row.find_all('td'))

[]